#### Con el afán de optimizar el funcionamiento de la API voy a crear datasets específicos para las funciones que lo necesiten.

In [1]:
import pandas as pd

In [2]:
steam_games = pd.read_csv('./datasets/steam_games.csv')
user_review = pd.read_csv('./datasets/user_reviews.csv')
user_items = pd.read_csv(f'./datasets/user_items_comp_csv.gz', compression='gzip')

### User For Genre

In [4]:
user_items['id'] = user_items['id'].astype(int)
generos = steam_games.drop(columns=['app_name','price','id','developer','Year','Accounting'])

# Unimos 2 datasets usando 'item_name' y 'app_name' como la clave
gener_user = pd.merge(user_items[['id','playtime_forever','user_id']],steam_games,left_on='id',right_on='id',how='inner')

# Inicializamos un arreglo vacío para almacenar los resultados
resultados_por_genero = []
for genero in generos.columns:
    # Filtramos las filas que corresponden al género actual
    genero_data = gener_user[gener_user[genero] == 1]

    # Comprobamos si existen filas que correspondan al género actual
    if not genero_data.empty:
        # Encontramos el usuario que tiene el valor maximo de 'playtime_forever' para el género actual
        user_max = genero_data.groupby('user_id')['playtime_forever'].sum().div(60).idxmax() # Divido por 60 ya que esta en minitus
        # Agrupamos los datos por año y calculamos la suma de 'playtime_forever'
        result_df = genero_data[genero_data['user_id'] == user_max].groupby('Year')['playtime_forever'].sum().reset_index()
        # Reformateamos los resultados
        formatted_result = [{'Año': record['Year'], 'Horas': record['playtime_forever']} for _, record in result_df.iterrows()]
        # Agregamos los resultados al arreglo
        resultados_por_genero.append({'Género': genero, 'Usuario': user_max, 'Año_Horas': formatted_result})
    else:
        # Si no existen filas que correspondan al género, solo incluimos el genero y el valor nulo para 'usuario' y un arreglo vacío para 'Año_Horas'
        resultados_por_genero.append({'Género': genero, 'Usuario': None, 'Año_Horas': []})
        
# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados_por_genero)

# Eliminamos las filas con valores faltantes
df_resultados.dropna(inplace=True)

# Guardamos los resultados en un archivo csv
df_resultados.to_csv('./Datasets/userForGenre.csv',index=False)